In [1]:
#pip install openprompt

In [1]:
import csv
csv.field_size_limit(1410720)

# 初始化字典
trec_dict = {}
q_dict = {}
query_id_counts = {}


# 从trec.txt中读取数据
with open('401-450.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        query_content = row[1]  # 第二列是查询内容
        trec_dict[query_id] = query_content

# 从lmdir.txt中读取数据
with open('401-450diri1000.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        doc_content = row[6]  # 第7列是文档内容
        # 计算当前查询id的文档数量
        if query_id_counts.get(query_id, 0) >= 5:
            continue
        
        
        # 在文档内容前添加标识
        # 将查询id和带标识的文档内容添加到字典中
        key = query_id + '\t' + trec_dict.get(query_id, '')
        i = len(q_dict.get(key, []))+1
        doc_content = f'Document {i} : {doc_content}'
        
        if key in q_dict:
            q_dict[key].append(doc_content)
        else:
            q_dict[key] = []
            q_dict[key].append(doc_content)
            
        query_id_counts[query_id] = query_id_counts.get(query_id, 0) + 1



# 打印出第一个查询id及其对应的文档内容，以验证代码是否正确
for query_id in list(q_dict.keys())[:1]:
    print("Query :", query_id)
    print("Document Contents:", q_dict[query_id])


Query : 401	What language and cultural differences impede the integration of foreign minorities in Germany?
Document Contents: ['Document 1 : FBIS4-18372 "drweu064_h_94001" FBIS-WEU-94-064 Daily Report 3 Apr 1994 GERMANY EU Minority Blocking Vote `Overestimated\' EU Minority Blocking Vote `Overestimated\' LD0304093094 Berlin DDP/ADN in German 0415 GMT 3 Apr 94 LD0304093094 Berlin DDP/ADN German BFN [Text] Nuremberg (DDP/ADN) -- In the view of Foreign Minister Klaus Kinkel, the practical significance of the minority blocking vote in the EU [European Union] Council of Ministers, which has until recently been strongly disputed because of the membership negotiations of the EFTA [European Free Trade Association] states, is being overestimated. In any case, in the EU the principle of unanimity governed complex matters, and the qualified majority vote was decisive for issues concerning the internal market, Kinkel told the DDP/ADN news agency in Nuremberg. Moreover, the EU foreign ministers ha

In [2]:
# q_dict['264014	how long is life cycle of flea']
type(q_dict)

dict

In [23]:
# from langchain.llms import OpenAI
# from langchain.chains.summarize import load_summarize_chain
# from langchain.chains import AnalyzeDocumentChain

# llm = OpenAI(openai_api_key="",temperature=0)


# def summarize_doc(doc):
#     # 处理文档的代码在这里
#     chain = load_summarize_chain(llm, chain_type="map_reduce")

#     summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=chain)

#     processed_doc = summarize_document_chain.run(doc)
      
#     return processed_doc


# for key, value in q_dict.items():
#     new_value = []
#     for doc in value:
#         new_value.append(summarize_doc(doc))
#     q_dict[key] = new_value


In [3]:
q_dict['401	What language and cultural differences impede the integration of foreign minorities in Germany?']

['Document 1 : FBIS4-18372 "drweu064_h_94001" FBIS-WEU-94-064 Daily Report 3 Apr 1994 GERMANY EU Minority Blocking Vote `Overestimated\' EU Minority Blocking Vote `Overestimated\' LD0304093094 Berlin DDP/ADN in German 0415 GMT 3 Apr 94 LD0304093094 Berlin DDP/ADN German BFN [Text] Nuremberg (DDP/ADN) -- In the view of Foreign Minister Klaus Kinkel, the practical significance of the minority blocking vote in the EU [European Union] Council of Ministers, which has until recently been strongly disputed because of the membership negotiations of the EFTA [European Free Trade Association] states, is being overestimated. In any case, in the EU the principle of unanimity governed complex matters, and the qualified majority vote was decisive for issues concerning the internal market, Kinkel told the DDP/ADN news agency in Nuremberg. Moreover, the EU foreign ministers had undertaken to reexamine the whole procedure, which has now been decided, in 1996. It had at any rate been right not to let th

In [4]:
import ast



# 从文件中读取字典
with open("q_dict401-450.txt", "r") as file:
    # 使用ast.literal_eval安全地将字符串转化为Python对象
    new_dict = ast.literal_eval(file.read())

# 遍历新读取的字典并进行修复
for key in list(new_dict.keys()):
    query_id, _ = key.split('\t')

    # 查找q_dict中是否有对应的查询ID
    corrected_key = next((k for k in q_dict if k.startswith(query_id + '\t')), None)

    if corrected_key:
        # 将新字典中的查询内容替换为q_dict中的查询内容
        new_dict[corrected_key] = new_dict.pop(key)

# 打印修复后的新字典以验证
print(new_dict)


{'401\tWhat language and cultural differences impede the integration of foreign minorities in Germany?': [' Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious.', ' Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945.', ' German Chancellor Helmut Kohl is in a confrontation with Turkey over arms

In [8]:
# 将修复后的new_dict保存为txt文件，使用utf-8编码
with open("corrected_q_dict401-450.txt", "w", encoding="utf-8") as file:
    file.write(str(new_dict))


In [8]:
[' Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious.',
 ' Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945.',
 ' German Chancellor Helmut Kohl is in a confrontation with Turkey over arms shipments, which must be used for defense within the NATO alliance. Hans-Ulrich Klose criticized Kohl for allowing the row to get out of hand. Meanwhile, the number of foreigners seeking political asylum in Germany has reached a record high of 35,000, with refugees from Yugoslavia, Romania, and Turkey making up the majority. In response to unrest in the Kurdish area, Turkey is introducing economic reforms to address the Kurdish problem.',
 ' The UK and Spain are seeking a legally binding protocol from Germany to give countries with 23-27 votes an automatic right to delay decisions, which could delay the entry of four new countries into the European Union. Greece has criticized the UK\'s stance as "blackmail of a tiny minority".',
 ' German Foreign Minister Klaus Kinkel spoke to the Bundestag in support of Russian President Yeltsin and the need for Russia to be integrated into the world economy. He also called for increased cultural exchange between European countries, with a focus on German language, and for more funds to be allocated to this purpose. He asked the Bundestag to provide the necessary support.']


In [1]:
import json
import ast

with open("q_dict401-450.txt", "r", encoding="utf-8") as file:
    q_dict = ast.literal_eval(file.read())



In [2]:
q_dict['401	What language and cultural differences impede the integration of foreign minorities in Germany?']

[' Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious.',
 ' Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945.',
 ' German Chancellor Helmut Kohl is in a confrontation with Turkey over arms shipments, which must be used for defense within the NATO alliance. Hans-Ulrich Klose criticized Kohl 

In [3]:
import itertools
from openprompt.data_utils import InputExample

input_examples = []

# 使用itertools.combinations来获取所有的两两组合，不会重复
for query_id_content in q_dict.keys():
    # 分割查询ID和查询内容
    query_id, query = query_id_content.split('\t', 1)
    documents = q_dict[query_id_content]
    
    # 构建InputExample
    input_example = InputExample(
        guid= len(input_examples),
        label= None,
        meta= {
            "queryid": query_id,
            "query": query,
            "documents": documents,
            #"querybid": queryb_id,
            #"queryb": queryb,
            #"documentsb": documentsb
        },
        text_a= "",
        text_b= "",
        tgt_text= query_id
    )
    
    # 将InputExample添加到列表中
    input_examples.append(input_example)


In [4]:
print(input_examples[2])

{
  "guid": 2,
  "label": null,
  "meta": {
    "documents": [
      " A new study has found that the drug etidronate, used to treat Paget's disease and a bone disorder in cancer patients, can halve the rate of vertebral fractures in women with osteoporosis. The drug is sold under the brand name Didronel and the manufacturer is applying to the FDA for approval to use the drug to treat osteoporosis. The study also found that the drug increased spinal bone density by an average of 4-5%. Further studies are needed to explore the drug's long-term effects and to compare its effectiveness to other osteoporosis treatments.",
      " Australian researchers have found that daughters of women with osteoporosis are more likely to develop the disease than those whose mothers do not have it, suggesting that genetics may be a factor in its development. The study compared 32 women with mothers who had osteoporosis to 22 women with mothers who did not, and found that the former had thinner bones.",
  

In [5]:
for example in input_examples:
    example.meta['documents'] = example.meta['documents'][0:2]
    #example.meta['documents'] = ""

In [6]:
for example in input_examples:
    # 对'documentsa'列表中的文档编上号并拼接
    documents = example.meta['documents']
    documents = [f'Document {i+1}: {doc}' for i, doc in enumerate(documents)]
    example.meta['documents'] = ' '.join(documents)

In [7]:
input_examples[2]

{
  "guid": 2,
  "label": null,
  "meta": {
    "documents": "Document 1:  A new study has found that the drug etidronate, used to treat Paget's disease and a bone disorder in cancer patients, can halve the rate of vertebral fractures in women with osteoporosis. The drug is sold under the brand name Didronel and the manufacturer is applying to the FDA for approval to use the drug to treat osteoporosis. The study also found that the drug increased spinal bone density by an average of 4-5%. Further studies are needed to explore the drug's long-term effects and to compare its effectiveness to other osteoporosis treatments. Document 2:  Australian researchers have found that daughters of women with osteoporosis are more likely to develop the disease than those whose mothers do not have it, suggesting that genetics may be a factor in its development. The study compared 32 women with mothers who had osteoporosis to 22 women with mothers who did not, and found that the former had thinner bone

In [8]:
# 假设 input_examples 是你的数据集
text_list = []
for example in input_examples:
    #text = 'Given the following two queries: the first query is \"' + example.meta['querya'] + '\", and the second query is \"' + example.meta['queryb'] + '\". Compared to the first query, the second query is more {"mask"}.'
    #text = 'Given the following two queries and their respective top one document----the first query is \"' + example.meta['querya'] + '\" with its top one search result as \"' + example.meta['documentsa'] + '\", and the second query is \"' + example.meta['queryb'] + '\" with its top one search result as \"' + example.meta['documentsb'] + '\". Compared to the first query, the second query is more {"mask"}.'
    #text = 'Query \"'+  example.meta['query'] + '\" is {"mask"}.' 
    text = 'Given the query \"'+  example.meta['query'] + '\" and its top two search results \"'+  example.meta['documents'] + '\" , it can be inferred that the query is {"mask"}.'
    text_list.append(text)


In [9]:
print(text_list[0])
print(text_list[1])
print(text_list[2])

Given the query "What language and cultural differences impede the integration of foreign minorities in Germany?" and its top two search results "Document 1:  Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious. Document 2:  Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945." , it can be infe

In [10]:
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptForClassification
from openprompt import PromptDataLoader
from openprompt.plms.utils import TokenizerWrapper
from collections import defaultdict
import torch

classes = [ 
    "specific",
    "generic"
]
scores = defaultdict(float)

plm, tokenizer, model_config, WrapperClass = load_plm("t5","google/flan-t5-base")
#plm, tokenizer, model_config, WrapperClass = load_plm("albert", "albert-base-v2")
#plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-base")
#plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2-large")
#plm, tokenizer, model_config, WrapperClass = load_plm("opt", "facebook/opt-iml-max-1.3b")

x=0
for t in text_list:
    print("---------------------------------------------------------------")
    print(t)
    dataset = []
    dataset.append(input_examples[x])
    x = x + 1
    promptTemplate = ManualTemplate(
        text = t,
        tokenizer = tokenizer,
    )
    promptVerbalizer = ManualVerbalizer(
        classes = classes,
        label_words = {
            "specific":["specific","precise","exact","well-formed","clear"],
            "generic":["generic","general","common","universal","ambiguous", "ill-formed", "vague"]
        },
        tokenizer = tokenizer,
    )
    promptModel = PromptForClassification(
        template = promptTemplate,
        plm = plm,
        verbalizer = promptVerbalizer,
    )
    data_loader = PromptDataLoader(
        dataset = dataset,
        tokenizer = tokenizer,
        template = promptTemplate,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=512, decoder_max_length=3,batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False, truncate_method="tail"
    )
    promptModel.eval()
    with torch.no_grad():
        for batch in data_loader:
            logits = promptModel(batch)
            preds = torch.argmax(logits, dim = -1)
            probs = torch.softmax(logits, dim=-1)
          
        for i, prob in enumerate(probs):
            # 获取query的id
            query_id = batch.tgt_text[i]
            print(query_id)

            print(prob[classes.index('specific')],prob[classes.index('generic')])

            # 根据模型的预测概率给query加分
            scores[query_id] += prob[classes.index('specific')]
            
    


---------------------------------------------------------------
Given the query "What language and cultural differences impede the integration of foreign minorities in Germany?" and its top two search results "Document 1:  Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious. Document 2:  Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events 

tokenizing: 1it [00:00, 892.41it/s]


401
tensor(0.1318) tensor(0.8682)
---------------------------------------------------------------
Given the query "What is happening in the field of behavioral genetics, the study of the relative influence of genetic and environmental factors on an individual's behavior or personality?" and its top two search results "Document 1:  A landmark study on identical twins raised apart has found that genetics play a much larger role in shaping behavior than family environment. The study, published in the journal Science, found that 70% of intelligence quotient, 50% of personality differences, 50% of religiosity, and 40% of job interest variations are accounted for by genes. The Minnesota study of identical twins, begun in 1979, has revealed many eerie similarities between reunited twins and has been able to quantify the contribution of genetics to personality and behavior, while also emphasizing the importance of environment in child development. Document 2:  Susan Hodge, a biomathemetician a

tokenizing: 1it [00:00, 500.04it/s]


402
tensor(0.1650) tensor(0.8350)
---------------------------------------------------------------
Given the query "Find information on the effects of the dietary intakes of potassium, magnesium and fruits and vegetables as determinants of bone mineral density in elderly men and women thus preventing osteoporosis (bone decay)." and its top two search results "Document 1:  A new study has found that the drug etidronate, used to treat Paget's disease and a bone disorder in cancer patients, can halve the rate of vertebral fractures in women with osteoporosis. The drug is sold under the brand name Didronel and the manufacturer is applying to the FDA for approval to use the drug to treat osteoporosis. The study also found that the drug increased spinal bone density by an average of 4-5%. Further studies are needed to explore the drug's long-term effects and to compare its effectiveness to other osteoporosis treatments. Document 2:  Australian researchers have found that daughters of women wi

...

448
tensor(0.1999) tensor(0.8001)
---------------------------------------------------------------
Given the query "What has caused the current ineffectiveness of antibiotics against infections and what is the prognosis for new drugs?" and its top two search results "Document 1:  Calgene conducted experiments to determine if APH(3 隆炉)II in fresh tomatoes could render orally-administered kanamycin ineffective. The results showed that APH(3 隆炉)II would be degraded before it could inactivate the antibiotics. The FDA evaluated the data and concluded that any active APH(3 隆炉)II would not significantly inactivate kanamycin or neomycin due to the small amount of ATP in fruits and vegetables. Therefore, the presence of alkaline phosphatase in food will not affect the therapeutic use of orally administered kanamycin or neomycin. Document 2:  Merck & Co, the world's largest pharmaceuticals group, has warned that European governments' efforts to cut drugs bills are ineffective and counter-producti

tokenizing: 1it [00:00, 999.83it/s]


449
tensor(0.0880) tensor(0.9120)
---------------------------------------------------------------
Given the query "How significant a figure over the years was the late Jordanian King Hussein in furthering peace in the Middle East?" and its top two search results "Document 1:  King Hussein of Jordan expresses his frustration and fears for the Middle East peace process, citing the lack of unity among Arab nations and the failure to coordinate with Israel. He expresses his disappointment at the opportunities missed and foreboding about the future, and his reluctance to sign a separate peace deal with Israel. He will travel to Washington to meet with President Bill Clinton and receive the results of a medical test, but is unable to summon much optimism about the health of the Arab world's body politic. Relations with Egypt remain cool and the king was snubbed by King Fahd during a recent trip to Saudi Arabia, though the favour of western allies is returning slowly. Document 2:  Palestinian

tokenizing: 1it [00:00, 499.74it/s]


450
tensor(0.4287) tensor(0.5713)


In [11]:
scores

defaultdict(float,
            {'401': tensor(0.1318),
             '402': tensor(0.1650),
             '403': tensor(0.2084),
             '404': tensor(0.2579),
             '405': tensor(0.1318),
             '406': tensor(0.0978),
             '407': tensor(0.2358),
             '408': tensor(0.2000),
             '409': tensor(0.2518),
             '410': tensor(0.1585),
             '411': tensor(0.2154),
             '412': tensor(0.1432),
             '413': tensor(0.0841),
             '414': tensor(0.1803),
             '415': tensor(0.2231),
             '416': tensor(0.3618),
             '417': tensor(0.0755),
             '418': tensor(0.2076),
             '419': tensor(0.1450),
             '420': tensor(0.1811),
             '421': tensor(0.1142),
             '422': tensor(0.2283),
             '423': tensor(0.2475),
             '424': tensor(0.1991),
             '425': tensor(0.1039),
             '426': tensor(0.1883),
             '427': tensor(0.1901),
         

In [12]:
sorted_queries = sorted(scores.items(), key=lambda item: item[1], reverse=True)


In [13]:
sorted_queries 

[('450', tensor(0.4287)),
 ('416', tensor(0.3618)),
 ('404', tensor(0.2579)),
 ('409', tensor(0.2518)),
 ('423', tensor(0.2475)),
 ('407', tensor(0.2358)),
 ('422', tensor(0.2283)),
 ('415', tensor(0.2231)),
 ('443', tensor(0.2202)),
 ('411', tensor(0.2154)),
 ('403', tensor(0.2084)),
 ('418', tensor(0.2076)),
 ('433', tensor(0.2058)),
 ('432', tensor(0.2000)),
 ('408', tensor(0.2000)),
 ('448', tensor(0.1999)),
 ('424', tensor(0.1991)),
 ('427', tensor(0.1901)),
 ('426', tensor(0.1883)),
 ('420', tensor(0.1811)),
 ('414', tensor(0.1803)),
 ('435', tensor(0.1795)),
 ('428', tensor(0.1793)),
 ('447', tensor(0.1744)),
 ('441', tensor(0.1651)),
 ('402', tensor(0.1650)),
 ('438', tensor(0.1648)),
 ('434', tensor(0.1621)),
 ('431', tensor(0.1606)),
 ('410', tensor(0.1585)),
 ('429', tensor(0.1486)),
 ('419', tensor(0.1450)),
 ('412', tensor(0.1432)),
 ('446', tensor(0.1424)),
 ('445', tensor(0.1338)),
 ('436', tensor(0.1336)),
 ('437', tensor(0.1328)),
 ('405', tensor(0.1318)),
 ('401', ten

In [14]:
query_dict = dict(sorted_queries)

In [15]:
query_dict = {k: v.item() for k, v in query_dict.items()}

In [16]:
query_dict

{'450': 0.4287034273147583,
 '416': 0.36184513568878174,
 '404': 0.25793835520744324,
 '409': 0.2518233060836792,
 '423': 0.24748799204826355,
 '407': 0.23584604263305664,
 '422': 0.22828592360019684,
 '415': 0.22306105494499207,
 '443': 0.22021852433681488,
 '411': 0.21540206670761108,
 '403': 0.20841513574123383,
 '418': 0.20759256184101105,
 '433': 0.2057809829711914,
 '432': 0.20003096759319305,
 '408': 0.2000299096107483,
 '448': 0.1999407261610031,
 '424': 0.19910505414009094,
 '427': 0.1900556981563568,
 '426': 0.18832719326019287,
 '420': 0.1810791790485382,
 '414': 0.18027493357658386,
 '435': 0.17954571545124054,
 '428': 0.1793217658996582,
 '447': 0.17437800765037537,
 '441': 0.1650732010602951,
 '402': 0.16503463685512543,
 '438': 0.16477301716804504,
 '434': 0.16207660734653473,
 '431': 0.16060727834701538,
 '410': 0.15852242708206177,
 '429': 0.14855147898197174,
 '419': 0.14499732851982117,
 '412': 0.14319583773612976,
 '446': 0.14235107600688934,
 '445': 0.1337763518095

In [17]:
with open('401-450gt.txt', 'r') as f:
    gt = dict(line.strip().split() for line in f)

with open('401-450nqc.txt', 'r') as f:
    nqc = dict(line.strip().split() for line in f)

query_scores = list(query_dict.values())
#gtscores = [float(gt[key]) for key in query_dict.keys()]
#nqcscores = [float(nqc[key]) for key in query_dict.keys()]


In [18]:
keys = query_dict.keys()
gtscores = [float(gt[key]) for key in keys]
nqcscores = [float(nqc[key]) for key in keys]


In [19]:
keys

dict_keys(['450', '416', '404', '409', '423', '407', '422', '415', '443', '411', '403', '418', '433', '432', '408', '448', '424', '427', '426', '420', '414', '435', '428', '447', '441', '402', '438', '434', '431', '410', '429', '419', '412', '446', '445', '436', '437', '405', '401', '430', '442', '439', '444', '421', '425', '440', '406', '449', '413', '417'])

In [20]:
with open('flant5_point_post2.txt', 'w', encoding='utf-8') as f:
    for key in query_dict.keys():
        f.write(f'{key}\t{query_dict[key]}\t{nqc[key]}\t{gt[key]}\n')


In [21]:
nqcscores

[13.707488059997559,
 -14.283669471740723,
 3.8539679050445557,
 9.889849662780762,
 31.011184692382812,
 7.814405918121338,
 3.7995567321777344,
 14.683598518371582,
 0.7333466410636902,
 8.253385543823242,
 45.91432189941406,
 2.5999250411987305,
 1.462353229522705,
 3.6750802993774414,
 6.774903774261475,
 0.9927824139595032,
 1.8052459955215454,
 13.625859260559082,
 1.6846462488174438,
 20.67823600769043,
 -8.841760635375977,
 1.9212779998779297,
 4.180014610290527,
 4.2269487380981445,
 8.429100036621094,
 2.892881155014038,
 2.7874035835266113,
 1.5539339780807495,
 -6.610340118408203,
 9.120701789855957,
 8.541165351867676,
 3.981992721557617,
 -0.05628133937716484,
 2.6592535972595215,
 11.008926391601562,
 2.1707212924957275,
 2.51373553276062,
 8.07620906829834,
 0.5529524683952332,
 11.102095603942871,
 2.685452461242676,
 4.02252721786499,
 11.117552757263184,
 1.7216060161590576,
 -12.099804878234863,
 1.0883139371871948,
 8.005461692810059,
 7.3450188636779785,
 0.666447

In [24]:
from scipy.stats import pearsonr
p1 = pearsonr(query_scores, gtscores)
p2 = pearsonr(nqcscores, gtscores)
print(p1)
print(p2)

PearsonRResult(statistic=0.08046093453046868, pvalue=0.5785825498021564)
PearsonRResult(statistic=0.362217280059528, pvalue=0.009740338185571466)


In [25]:
from scipy.stats import kendalltau
k1 = kendalltau(query_scores, gtscores)
k2 = kendalltau(nqcscores, gtscores)
print(k1)
print(k2)

SignificanceResult(statistic=0.10040816326530612, pvalue=0.303537240020898)
SignificanceResult(statistic=0.23265306122448978, pvalue=0.01712632537811409)
